In [1]:
import numpy as np
import pandas as pd

movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

movies = movies.merge(credits,on='title')

#we are working with tags to find similarity between content so not considering many numerical factors 


#genres - 
#id - include if you want to add posters with movie names
#keywords 
#title
#overview
#cast 
#crew

movies = movies[['title','overview','genres','keywords','cast','crew']]

movies.isnull().sum()

movies.dropna(inplace=True)

import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj) :
        L.append(i['name'])
    return L

movies['genres']=movies['genres'].apply(convert)

movies['keywords'] = movies['keywords'].apply(convert)

#made a new function as we only want to keep the top 3 crew of every movie 

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj) :
        if counter != 3:
            L.append(i['name'])
            counter +=1
        else:
            break
    return L

movies['cast']= movies['cast'].apply(convert3)

def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj) :
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

movies['crew']= movies['crew'].apply(fetch_director)

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['overview'] = movies['overview'].apply(lambda x:x.split())

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


new = movies.drop(columns=['overview','genres','keywords','cast','crew'])


new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    

vector = cv.fit_transform(new['tags']).toarray()

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)

import pickle

pickle.dump(new,open('movies.pkl','wb'))

pickle.dump(new.to_dict(),open('movie_dict.pkl','wb'))

pickle.dump(similarity,open('similarity.pkl','wb'))

C:\Users\garvi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
